<a href="https://colab.research.google.com/github/Akiwacky/Coefficient-of-Variation/blob/main/Coefficient_of_Variation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
""" A class used to pull the average rate of return for a ticker """

import pandas as pd
import requests
from datetime import datetime
import numpy as np

class AverageRateofReturn:
  ''' Program to calculate the Average annualized returns of a stock '''
  def __init__(self, ticker):
    self.ticker = ticker 
    self.API_KEY = "OKBZVRUEJ4LTZRDQ"
    self.data = {}

  def run_time_series(self):
    """ API to return monthly adjusted time series  """
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={self.ticker}&apikey={self.API_KEY}'
    re = requests.get(url)
    data = re.json()
    self.data = data['Monthly Adjusted Time Series']
    return self.data
  
  def get_business_date(self, year, month, day=1):
    """ Returns the last business day of the month
        Parameters:
        year(int): year requested
        month(int): month requested
        day(int): default to 1
    
     """
    start = datetime(year, month, day)
    index = pd.date_range(start=start, periods=1, freq="BM")
    date = index[0].strftime("%Y-%m-%d")
    return date
  
  def stock_return_calculator(self, start_month, start_year, end_month, end_year):
    """ Returns annualized return for requested stock 
        Parameters:
        start_month (int): Start Month
        start_year (int): Start Year
        end_month (int): End Month
        end_year (int): End Year
    """

    if self.data == {}:
      self.run_time_series()
    start_date = float(self.data[self.get_business_date(start_year, start_month)]['5. adjusted close'])
    end_date = float(self.data[self.get_business_date(end_year, end_month)]['5. adjusted close'])
    if start_year == end_year:
      result = round(((end_date / start_date)-1)*100,2)
    else:
      result = round((((end_date / start_date)** (1/(end_year-start_year))) - 1) * 100, 2)
    return result

  
  def calculate_year_returns(self, start_month, start_year, end_month, end_year):
    """ Collects ticker returns for years requested """
    lst = []
    for y in range(start_year, end_year, 1):
      lst.append(self.stock_return_calculator(start_month,y,end_month,y+1))
    lst.reverse()
    return lst
  
  def calculate_cv(self, lst):
    """ Calculates the coefficient of variation for the ticker """
    cv = np.std(lst, ddof=1) / np.mean(lst) * 100
    return cv


In [148]:
SPY = AverageRateofReturn("SPY")
VTI = AverageRateofReturn("VTI")

In [149]:
spy = SPY.calculate_year_returns(2,2017,2,2022)
vti = VTI.calculate_year_returns(2, 2017, 2, 2022)
vti

[11.99, 35.31, 6.89, 5.2, 16.23]

In [154]:
VTI.calculate_cv(vti)

79.94818738568044

In [155]:
SPY.calculate_cv(spy)

65.59287546149365

##Visuals

In [156]:
import plotly.graph_objects as go

In [160]:
tickers=['SPY', 'VTI']

fig = go.Figure([go.Bar(x=tickers, y=[SPY.calculate_cv(spy), VTI.calculate_cv(vti)])])
fig.show()